In [21]:
import numpy as np
import matplotlib as plt
import time
from collections import Counter

In [72]:
#Size of data stream
m = 1000
#the k of F_k
k = 2
#two user defined parameters lambda and eplison, so that Pr(|Y-F_k|>lambda F_k) < eplison
lambd = 0.1
eplison = 0.1

In [79]:
def stream_distribution(mode,param1=0,param2=0):
    if mode=="uniform":
        return np.random.randint(low=0, high=param1, size=m)
    if mode=="Zipf":
        return np.random.zipf(param1, m)
    if mode=="Gaussian":
        return (np.floor(np.random.normal(loc=param1, scale=param2, size=m))).astype("int")
    
def online_search(stream):
        curr_r = 1
        curr_a = stream[0]
        for i in np.linspace(1,m-1,m-1):
            i = int(i)
            rd = np.random.uniform()
            if rd < 1/(i+1):
                curr_a = stream[i]
                curr_r = 1
            elif curr_a==stream[i]:
                curr_r = curr_r + 1
        
        return curr_r

In [80]:
#define a stream
a = stream_distribution("Gaussian",0,20)

In [81]:
#compute the ground truth
a_counter = Counter(a)
frequency_vector = np.array([a_counter[i] for i in sorted(list(a_counter))])
ground_true = np.sum(frequency_vector**k)

In [82]:
#the size of N
n = frequency_vector.shape[0]

In [95]:
# TODO: online update

class AMS_offline(object):
    '''
    AMS Sketch for offline learning. We precompute the frequency vector of the data
    stream. Online update is not implemented yet.
    '''

    def __init__(self, stream, lambd, eplison):
        '''
        Build frequency vector
        :param stream: list of non-negative integer in range(0, RANGE)
        '''
        counter = Counter(stream)
        self.frequency_vector = np.array([counter[i] for i in sorted(list(counter))])

        self.F_1 = np.sum(self.frequency_vector)
        
        #compute s1 and s2 in the paper
        self.s1 = int(np.ceil(1*k*n**(1-1/k)/lambd**2))
        self.s2 = int(np.ceil(2*np.log(1/eplison)))
        
        self.stream = stream

        

        
    def estimate_F2(self):
        '''
        Implement the improved estimation of F2 with matrix multiplication.
        Uniformly random hashing is used in place of the four-independent hashing
        This is not exactly the same as in the original paper, but is the most popular
        way for implementation nowadays
        :return: estimated F_2
        '''
        self.hashing_matrix = np.random.choice(a = [1, -1], size=(self.s1, self.s2, n), replace=True)
        Z = np.matmul(self.hashing_matrix, self.frequency_vector)
        X = np.square(Z)
        Y = np.mean(X, axis=0)
        estimation = np.median(Y)

        return estimation


    def estimate_Fk(self, k):
        '''
        Implement the estimation of Fk
        :return: estimated Fk
        '''
        self.X = np.zeros((self.s1, self.s2))

        prob = self.frequency_vector / self.F_1

        for i in range(self.s1):
            for j in range(self.s2):
                idx = np.random.choice(a=n, size=1, p=prob)[0]
                r = np.random.randint(low=1, high=self.frequency_vector[idx], size=1)[0]
                self.X[i][j] = self.F_1 * (r**k - (r - 1)**k)

        Y = np.mean(self.X, axis=0)
        estimation = np.median(Y)
        
        return estimation
    
    def estimate_Fk_vectorized(self, k):
        
        prob = self.frequency_vector / self.F_1
        idx = np.random.choice(a=n, size=(self.s1, self.s2), p=prob)

        onehot_idx = np.eye(n)[idx]
        freq = np.matmul(onehot_idx, self.frequency_vector)
        offset = np.random.random(size=(self.s1, self.s2))
        r = np.round((freq - 1) * offset) + 1


        X = (np.power(r, k) - np.power(r-1, k)) * self.F_1
        Y = np.mean(X, axis=0)
        estimation = np.median(Y)
        
        return estimation
    
    def estimate_FK_online(self, k):
        
        self.X = np.zeros((self.s1, self.s2))
        
        a = np.ones((self.s1, self.s2))*self.stream[0]
        r = np.ones((self.s1, self.s2))
        
        for i in np.linspace(1,m-1,m-1):
            i = int(i)
            
            r = r + (a == self.stream[i])
            
            rd = np.random.uniform(size=(self.s1, self.s2))
            replace = rd < 1/(i+1)
            remain = rd >= 1/(i+1)
            
            a = (a * remain) + (replace * self.stream[i])
            r = (r * remain) + (replace * 1)
            
        self.X = (np.power(r, k) - np.power(r-1, k)) * self.F_1
            

        Y = np.mean(self.X, axis=0)
        estimation = np.median(Y)
        
        return estimation
            

In [103]:
A = AMS_offline(a, lambd, eplison)
time1 = time.time()
AMS1 = A.estimate_F2()
time2 = time.time()
# AMS2 = A.estimate_Fk(2)
time3 = time.time()
AMS3 = A.estimate_Fk_vectorized(k)
time4 = time.time()
AMS4 = A.estimate_FK_online(k)
time5 = time.time()

print("The ground true value is:",ground_true)
print("F_2 function return AMS sketch:",AMS1.astype("float32"), "using",(time2-time1),"second")
# print("F_k function return AMS sketch:",AMS2.astype("float32"), "using",(time3-time2),"second")
print("Offline F_k Vec function return AMS sketch:",AMS3.astype("float32"), "using",(time4-time3),"second")
print("Online F_k Vec function return AMS sketch:",AMS4.astype("float32"), "using",(time5-time4),"second")

The ground true value is: 14484
F_2 function return AMS sketch: 14478.993 using 0.05405926704406738 second
Offline F_k Vec function return AMS sketch: 14653.283 using 0.035396575927734375 second
Online F_k Vec function return AMS sketch: 14587.151 using 0.959892988204956 second


In [104]:
###check if the result meet the accuracy
print("The F_2 function return meet the accuracy:", np.abs(AMS1-ground_true)<lambd*ground_true,". With error",np.abs(AMS1-ground_true)/ground_true)
# print("The F_k function return meet the accuracy:", np.abs(AMS2-ground_true)<lambd*ground_true,". With error",np.abs(AMS2-ground_true)/ground_true)
print("The Offline F_k function return meet the accuracy:", np.abs(AMS3-ground_true)<lambd*ground_true,". With error",np.abs(AMS3-ground_true)/ground_true)
print("The Online F_k function return meet the accuracy:", np.abs(AMS4-ground_true)<lambd*ground_true,". With error",np.abs(AMS4-ground_true)/ground_true)

The F_2 function return meet the accuracy: True . With error 0.0003456977007971546
The Offline F_k function return meet the accuracy: True . With error 0.011687582682084478
The Online F_k function return meet the accuracy: True . With error 0.007121763992310169
